In [1]:
import joblib
import json
import pandas as pd
import numpy as np
import tensorflow as tf
#import tensorflowjs as tfjs
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
trainDtf = pd.read_csv(r"dataset_train.csv")
valDtf = pd.read_csv(r"dataset_validation.csv")
testDtf = pd.read_csv(r"dataset_test.csv")

In [3]:
x_train = trainDtf[["MathScore","ReadingScore","WritingScore"]].to_numpy()
y_train = trainDtf["WklyStudyHours"].to_numpy() 

x_val = valDtf[["MathScore","ReadingScore","WritingScore"]].to_numpy()
y_val = valDtf["WklyStudyHours"].to_numpy()

x_test = testDtf[["MathScore","ReadingScore","WritingScore"]].to_numpy()
y_test = testDtf["WklyStudyHours"].to_numpy()

print(len(x_train))
print(len(x_val))
print(len(x_test))

24512
3065
3064


In [4]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

joblib.dump(label_encoder, 'label_encoder.pkl')

label_mapping = {label: index for index, label in enumerate(label_encoder.classes_)}
with open('label_encoder.json', 'w') as f:
    json.dump(label_mapping, f)

print(y_train_encoded)
print(y_val_encoded)
print(y_test_encoded)

[1 2 1 ... 2 0 0]
[1 0 0 ... 0 1 1]
[1 1 0 ... 0 1 2]


In [5]:
num_classes = len(np.unique(y_train))
print(num_classes)
print(x_train.shape[1])
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
# tf.keras.optimizers.RMSprop(learning_rate=0.0000001)
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

3
3




In [6]:
history = model.fit(x=x_train, y=y_train_encoded, validation_data=(x_val, y_val_encoded), epochs=27, batch_size=32)

Epoch 1/27


766/766 [==============================] - 2s 1ms/step - loss: 1.0722 - accuracy: 0.5087 - val_loss: 1.0070 - val_accuracy: 0.5491
Epoch 2/27
766/766 [==============================] - 1s 1ms/step - loss: 1.0142 - accuracy: 0.5387 - val_loss: 0.9888 - val_accuracy: 0.5491
Epoch 3/27
766/766 [==============================] - 1s 1ms/step - loss: 1.0008 - accuracy: 0.5462 - val_loss: 0.9959 - val_accuracy: 0.5491
Epoch 4/27
766/766 [==============================] - 1s 1000us/step - loss: 0.9941 - accuracy: 0.5470 - val_loss: 0.9890 - val_accuracy: 0.5491
Epoch 5/27
766/766 [==============================] - 1s 992us/step - loss: 0.9919 - accuracy: 0.5474 - val_loss: 0.9884 - val_accuracy: 0.5491
Epoch 6/27
766/766 [==============================] - 1s 998us/step - loss: 0.9920 - accuracy: 0.5474 - val_loss: 0.9904 - val_accuracy: 0.5491
Epoch 7/27
766/766 [==============================] - 1s 986us/step - loss: 0.9912 - accuracy: 0.5474 - val_loss: 0.9902 - val_accuracy: 0.

In [7]:
results = model.evaluate(x_test, y_test_encoded, verbose=1)

96/96 [==============================] - 0s 784us/step - loss: 0.9976 - accuracy: 0.5441


In [8]:
model.save('wklyStudyHours_model.h5')
model.save('wklyStudyHours_model.keras', save_format='keras')
model.save("wklyStudyHours_SavedModel", save_format='tf')
#tfjs.converters.save_keras_model(model, "wklyStudyHours_TensorFlowJS")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("wklyStudyHours.tflite", "wb") as f:
    f.write(tflite_model)

c:\Users\Ryannnnnnnn\anaconda3\envs\tf\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: wklyStudyHours_SavedModel\assets


INFO:tensorflow:Assets written to: wklyStudyHours_SavedModel\assets


INFO:tensorflow:Assets written to: C:\Users\RYANNN~1\AppData\Local\Temp\tmp_4xjroc3\assets


INFO:tensorflow:Assets written to: C:\Users\RYANNN~1\AppData\Local\Temp\tmp_4xjroc3\assets
